# BA2A - <u> Implement MotifEnumeration

In [ ]:
################################ HELPER FUNCTIONS #####################################
import itertools

def all_combinations(k):
  return [''.join(p) for p in itertools.product("ACGT", repeat=k)]

def hamming_dist(kmer1, kmer2):
  return sum(i!=j for i,j in zip(kmer1,kmer2))

def kmer_exists_with_mismatch(kmer, dna,d):
  return any(hamming_dist(kmer, dna[i : i+len(kmer)])<=d for i in range(len(dna)-len(kmer)+1))


def kmer_set_from_dna_list(dna_list, k):
  return {dna_list[i][j: j+k] for i in range(len(dna_list)) for j in range(len(dna_list[i]) -k +1)}

def neighbourhood_kmer(all_combo, kmer,d):
  for nkmer in all_combo:
    if hamming_dist(nkmer, kmer) <= d:
      yield nkmer

#######################################################################################

# MY IMPLEMENTATION
def motif_enumerate(dna_list, k, d):
  motif_set = set()

  for kmer in all_combinations(k):
     if all(kmer_exists_with_mismatch(kmer,dna, d) for dna in dna_list):
       motif_set.add(kmer)

  return ' '.join(motif_set)


# ACCORDING TO GIVEN ALGORITHM
def motif_enumeration(dna, k, d):
  final_motif_set = set()
  all_kmer_set = kmer_set_from_dna_list(dna_list, k)
  all_combo = all_combinations(k)

  for kmer in all_kmer_set:
    neighbour_kmer_list = neighbourhood_kmer(all_combo, kmer,d)

    for neighbour_kmer in neighbour_kmer_list:
      if all(kmer_exists_with_mismatch(neighbour_kmer, dna, d) for dna in dna_list):
        final_motif_set.add(neighbour_kmer)

  return ' '.join(final_motif_set)

In [ ]:
# DRIVER CODE
dna1 = 'ATTTGGC'
dna2 = 'TGCCTTA'
dna3 = 'CGGTATC'
dna4 = 'GAAAATT'
k, d = 3, 1
dna_list = [dna1, dna2, dna3, dna4]
            
motif_enumeration(dna_list, k, d)

'TTT ATA GTT ATT'

<hr>

# BA2B - <u> Find a Median String

In [ ]:
################################ HELPER FUNCTIONS #####################################
import itertools

def hamming_dist(kmer1, kmer2):
  return sum(i!=j for i,j in zip(kmer1,kmer2))

def kmer_set(dna, k):
  return {dna[i:i+k]for i in range(len(dna)-k+1)}
  
def all_combinations(k):
  return [''.join(p) for p in itertools.product("ACGT", repeat=k)]
  
def min_score_from_single_dna(pattern, dna):
  return min(hamming_dist(pattern, kmer) for kmer in kmer_set(dna, len(pattern)))

def min_score_from_all_dna(pattern, dna_list):
  return sum(min_score_from_single_dna(pattern, dna) for dna in dna_list)

########################################################################################

def find_median_string(k, dna_list):

  # MINIMUM DISTANCE INITIALIZATION
  min_dist = k*len(dna_list)

  all_combo = all_combinations(k)

  for kmer in all_combo:
    cur_dist = min_score_from_all_dna(kmer, dna_list)
    
    if cur_dist <= min_dist:
      min_dist = cur_dist
      motif = kmer
    
  return motif


In [ ]:
# DRIVER CODE
k = 6
dna1 = 'GCCGCAGATGTAAGCGTTAAACGCGGGGATGCTAGCCGAGGG'
dna2 = 'ACCATTCCTCCAATCAACGCTACCAAATTTGGTAGTCGGGCA'
dna3 = 'GCTACCTTGAAATCAGGGTAGGGTAACCCAACGCCAGTCATG'
dna4 = 'AGCTGCTGAGCCAATAGAGCTACCCGCGCTAAATAAAGCCCC'
dna5 = 'CGACTGTATGTCCAGCGACAGGGTGCTAGCGCAACCATAATG'
dna6 = 'GGGAGCGCGACCACCGAGACCCTACGTAGGTGAACTGCTAAC'
dna7 = 'TCCAACTTTATCTAAGTCAGAGGTAGCATCATAGAGGCTAAC'
dna8 = 'TGACATAATGATGAAGATGCTATCTTGACTATTTGCCTGTGG'
dna9 = 'ACGATGGACCGGAGGAGTGCTATCCGTCAGTAATTCGGGTTA'
dna10 = 'CTTCCGCTTACACTTACTCTATGCGGTGTCGAGCCTGCTACC'


dna_list = [dna1, dna2, dna3, dna4, dna5, dna6, dna7, dna8, dna9, dna10]

find_median_string(k, dna_list)

'GCTACC'

<hr>

# BA2C - <u> Find a Profile-most Probable k-mer in a String

In [ ]:
####################################### HELPER FUNCTIONS #############################
import math
import numpy as np

def kmer_set(dna, k):
  return {dna[i:i+k] for i in range(len(dna)-k+1)}

def create_profile_dict(profile_str):
  return {'ACGT'[idx] : [float(col) for col in row.split()] for idx, row in enumerate(profile_str.split('\n'))}

def calculate_probabilty(kmer, profile_dict):
  return np.prod([profile_dict[nucleotide][i] for i, nucleotide in enumerate(kmer)])

#####################################################################################

def profile_most_probable_kmer(text, k, profile_dict):
  return max([(calculate_probabilty(kmer, profile_dict), kmer) for kmer in kmer_set(text, k)], key =lambda item : item[0])[1]


In [ ]:
# DRIVER CODE
text = 'TACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
k = 5
profile_str = '''0.2 0.2 0.3 0.2 0.3
                  0.4 0.3 0.1 0.5 0.1
                  0.3 0.3 0.5 0.2 0.4
                  0.1 0.2 0.1 0.1 0.2'''


profile_dict = create_profile_dict(profile_str)
profile_most_probable_kmer(text, k, profile_dict)

'AGCACCG'

<hr>

# BA2D - <u> Implement GreedyMotifSearch

In [ ]:
############################################### HELPER FUNCTIONS #################################
from collections import defaultdict, Counter
import numpy as np

def kmer_list(dna, k):
  return [dna[i:i+k] for i in range(len(dna)-k+1)]

def calculate_probabilty(kmer, profile_dict):
  return np.prod([profile_dict[nucleotide][i] for i, nucleotide in enumerate(kmer)])

def profile_most_probable_kmer(text, k, profile_dict):
  return max([(calculate_probabilty(kmer, profile_dict), kmer) for kmer in kmer_list(text, k)], key =lambda item : item[0])[1]

def get_column(dna_list, col_num=0):
  return [dna[col_num] for dna in dna_list]

def create_count_dict(dna_list, k):
  nucleotides = 'ACGT'
  count_dict = defaultdict(list)

  for i in range(k):
      column = get_column(dna_list, col_num = i)

      for n in nucleotides:
        count_dict[n].append(column.count(n))
  
  return count_dict


def create_profile_dict(dna_list, k, t):
  count_dict = create_count_dict(dna_list, k)
  return {key: [val/t for val in count_dict[key]] for key in count_dict}

def calculate_score(motif_list, k, t):
  return sum(t-Counter(get_column(motif_list, col_num = i)).most_common(1)[0][1] for i in range(k))

################################################################################


def greedy_motif_search(dna_list, k, t):
  best_motif_list = [dna[:k] for dna in dna_list]
  

  for kmer in kmer_list(dna_list[0], k):
    cur_motif_list = [kmer]

    for i in range(1,t):
      profile_dict = create_profile_dict(cur_motif_list, k, t)
      cur_motif_list.append(profile_most_probable_kmer(dna_list[i], k,  profile_dict))

    if  calculate_score(cur_motif_list, k, t) < calculate_score(best_motif_list, k, t):
      best_motif_list = cur_motif_list


  return "\n".join(best_motif_list)


In [ ]:
# DRIVER CODE
dna1= "GGCGTTCAGGCA"
dna2= "AAGAATCAGTCA"
dna3= "CAAGGAGTTCGC"
dna4= "CACGTCAATCAC"
dna5= "CAATAATATTCG"

k, t = 3, 5
dna_list = [dna1, dna2, dna3, dna4, dna5]

print(greedy_motif_search(dna_list, k, t))

CAG
CAG
CAA
CAA
CAA


<hr>

# BA2E - <u> Implement GreedyMotifSearch with Pseudocounts

In [ ]:
############################################### HELPER FUNCTIONS #################################
from collections import defaultdict, Counter
import numpy as np

def kmer_list(dna, k):
  return [dna[i:i+k] for i in range(len(dna)-k+1)]

def calculate_probabilty(kmer, profile_dict):
  return np.prod([profile_dict[nucleotide][i] for i, nucleotide in enumerate(kmer)])

def profile_most_probable_kmer(text, k, profile_dict):
  return max([(calculate_probabilty(kmer, profile_dict), kmer) for kmer in kmer_list(text, k)], key =lambda item : item[0])[1]

def get_column(dna_list, col_num=0):
  return [dna[col_num] for dna in dna_list]

def create_pseudocount_dict(dna_list):
  nucleotides = 'ACGT'
  count_dict = defaultdict(list)

  for i in range(len(dna_list[0])):
      column = get_column(dna_list, col_num = i)

      for n in nucleotides:
        count_dict[n].append(column.count(n) + 1)
  
  return count_dict


def create_profile_dict(dna_list):
  count_dict = create_pseudocount_dict(dna_list)
  ln = len(dna_list)
  return {key: [val/(ln+4) for val in count_dict[key]] for key in count_dict}

def calculate_score(motif_list, k, t):
  return sum(t-Counter(get_column(motif_list, col_num = i)).most_common(1)[0][1] for i in range(k))

################################################################################


def greedy_motif_search_with_pseudo(dna_list, k, t):
  best_motif_list = [dna[:k] for dna in dna_list]
  
  for kmer in kmer_list(dna_list[0], k):
    cur_motif_list = [kmer]

    for i in range(1,t):
      profile_dict = create_profile_dict(cur_motif_list)
      cur_motif_list.append(profile_most_probable_kmer(dna_list[i], k,  profile_dict))

    if  calculate_score(cur_motif_list, k, t) < calculate_score(best_motif_list, k, t):
      best_motif_list = cur_motif_list


  return "\n".join(best_motif_list)


In [ ]:
# DRIVER CODE
dna1= "GGCGTTCAGGCA"
dna2= "AAGAATCAGTCA"
dna3= "CAAGGAGTTCGC"
dna4= "CACGTCAATCAC"
dna5= "CAATAATATTCG"

k, t = 3, 5
dna_list = [dna1, dna2, dna3, dna4, dna5]


print(greedy_motif_search_with_pseudo(dna_list, k, t))

TTC
ATC
TTC
ATC
TTC


<hr>

# BA2F - <u> Implement RandomizedMotifSearch


In [26]:
############################################### HELPER FUNCTIONS #################################
from collections import defaultdict, Counter
import numpy as np
import random

def select_motifs_random(dna_list, k):
  ln = len(dna_list[0])

  return [ dna[random:random+k] for dna,random in zip(dna_list, [random.randint(0, ln-k) for _ in dna_list])]

def kmer_list(dna, k):
  return [dna[i:i+k] for i in range(len(dna)-k+1)]

def calculate_probabilty(kmer, profile_dict):
  return np.prod([profile_dict[nucleotide][i] for i, nucleotide in enumerate(kmer)])

def profile_most_probable_kmer(text, k, profile_dict):
  return max([(calculate_probabilty(kmer, profile_dict), kmer) for kmer in kmer_list(text, k)], key =lambda item : item[0])[1]

def get_column(dna_list, col_num=0):
  return [dna[col_num] for dna in dna_list]

def create_pseudocount_dict(dna_list):
  nucleotides = 'ACGT'
  count_dict = defaultdict(list)

  for i in range(len(dna_list[0])):
      column = get_column(dna_list, col_num = i)

      for n in nucleotides:
        count_dict[n].append(column.count(n) + 1)
  
  return count_dict


def create_profile_dict(dna_list):
  count_dict = create_pseudocount_dict(dna_list)
  ln = len(dna_list)
  return {key: [val/(ln+4) for val in count_dict[key]] for key in count_dict}

def calculate_score(motif_list, k, t):
  return sum(t-Counter(get_column(motif_list, col_num = i)).most_common(1)[0][1] for i in range(k))

################################################################################


def randomized_motif_search_with_pseudo(dna_list, k, t):
  best_motif_list = select_motifs_random(dna_list, k)
  best_motif_score = calculate_score(best_motif_list, k, t)

  while True:
    profile_dict = create_profile_dict(best_motif_list)
    cur_motif_list = [profile_most_probable_kmer(dna, k,  profile_dict) for dna in dna_list]

    cur_motif_score = calculate_score(cur_motif_list, k, t)
    
    if cur_motif_score < best_motif_score:
      best_motif_list = cur_motif_list
      best_motif_score = cur_motif_score
    else:
      break

  return ("\n".join(best_motif_list), best_motif_score)




In [37]:
k,t = 8, 5
dna_1 ='CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA'
dna_2 = 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG'
dna_3 = 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT'
dna_4 ='TAGATCAAGTTTCAGGTGCACGTCGGTGAACC'
dna_5 ='AATCCACCAGCTCCACGTGCAATGTTGGCCTA'

dna_list = [dna_1, dna_2, dna_3, dna_4, dna_5]

best_motifs, best_score = None, 7000

for i in range(1000):
  cur_motifs , cur_score = randomized_motif_search_with_pseudo(dna_list, k, t)

  if cur_score < best_score:
    best_motifs = cur_motifs
    best_score = cur_score

print(best_motifs)
print(best_score)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG
9


<hr>

# BA2G - <u> Implement GibbsSampler


In [34]:
############################################### HELPER FUNCTIONS #################################
from collections import defaultdict, Counter
import numpy as np
import random

def select_motifs_random(dna_list, k):
  ln = len(dna_list[0])

  return [ dna[random:random+k] for dna,random in zip(dna_list, [random.randint(0, ln-k) for _ in dna_list])]

def kmer_list(dna, k):
  return [dna[i:i+k] for i in range(len(dna)-k+1)]

def calculate_probabilty(kmer, profile_dict):
  return np.prod([profile_dict[nucleotide][i] for i, nucleotide in enumerate(kmer)])

def profile_randomly_generated_kmer(dna, k, profile_dict):
  kmers = kmer_list(dna, k)
  probability_distribution = [calculate_probabilty(kmer, profile_dict) for kmer in kmers]


  return random.choices( population = kmers, weights=probability_distribution, k=1)[0]

def get_column(dna_list, col_num=0):
  return [dna[col_num] for dna in dna_list]

def create_pseudocount_dict(dna_list):
  nucleotides = 'ACGT'
  count_dict = defaultdict(list)

  for i in range(len(dna_list[0])):
      column = get_column(dna_list, col_num = i)

      for n in nucleotides:
        count_dict[n].append(column.count(n) + 1)
  
  return count_dict


def create_profile_dict(dna_list):
  count_dict = create_pseudocount_dict(dna_list)
  ln = len(dna_list)
  return {key: [val/(ln+4) for val in count_dict[key]] for key in count_dict}

def calculate_score(motif_list, k, t):
  return sum(t-Counter(get_column(motif_list, col_num = i)).most_common(1)[0][1] for i in range(k))

################################################################################

def gibbssampler_motif_search_with_pseudo(dna_list, k, t, N):
  best_motif_list = select_motifs_random(dna_list, k)
  best_motif_list_score = calculate_score(best_motif_list, k, t)
  
  for j in range(N):
    i = random.randint(0,t-1)

    profile_dict = create_profile_dict([best_motif_list[idx] for idx in range(t) if idx != i])

    cur_motif_list = [motif for motif in best_motif_list]
    cur_motif_list[i] = profile_randomly_generated_kmer(dna_list[i],k, profile_dict)
    cur_motif_list_score = calculate_score(cur_motif_list, k, t)

    
    if cur_motif_list_score < best_motif_list_score:
      best_motif_list = cur_motif_list
      best_motif_list_score = cur_motif_list_score

  return ("\n".join(best_motif_list), best_motif_list_score)


In [35]:
# DRIVER CODE

k, t, N = 8, 5, 100
dna_1 = 'CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA'
dna_2 ='GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG'
dna_3 ='TAGTACCGAGACCGAAAGAAGTATACAGGCGT'
dna_4 ='TAGATCAAGTTTCAGGTGCACGTCGGTGAACC'
dna_5 = 'AATCCACCAGCTCCACGTGCAATGTTGGCCTA'

dna_list = [dna_1, dna_2, dna_3, dna_4, dna_5]

best_motifs, best_score = None, 10000

for i in range(60):
  cur_motifs , cur_score =gibbssampler_motif_search_with_pseudo(dna_list, k, t, N)

  if cur_score < best_score:
    best_motifs = cur_motifs
    best_score = cur_score

print(best_motifs)
print(best_score)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG
9


<hr>

# BA2H - <u> Implement DistanceBetweenPatternAndStrings

In [40]:
################################ HELPER FUNCTIONS #####################################
import itertools

def hamming_dist(kmer1, kmer2):
  return sum(i!=j for i,j in zip(kmer1,kmer2))

def kmer_set(dna, k):
  return {dna[i:i+k]for i in range(len(dna)-k+1)}
  
def min_dist_with_single_string(pattern, dna):
  return min(hamming_dist(pattern, kmer) for kmer in kmer_set(dna, len(pattern)))

########################################################################################

def dist_between_pattern_and_strings(pattern, string_list):
  return sum(min_dist_with_single_string(pattern, string) for string in string_list)

In [41]:
# DRIVER CODE
pattern = 'AAA'
string_list = ['TTACCTTAAC', 'GATATCTGTC', 'ACGGCGTTCG', 'CCCTAAAGAG', 'CGTCAGAGGT']


min_score_from_all_dna(pattern , string_list)

5

<hr>